In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import polars as pl
df = pl.read_parquet(r"C:\Users\Ashle\OneDrive\Documents\bet_edge\data\es_cont_ohlcv_20160601_to_20250108.parquet")

In [17]:
from bet_edge.backtesting.strategies.ma_cross_macd_strategy import MACrossMACDStrategy
from bet_edge.backtesting.portfolio import Portfolio
from bet_edge.backtesting.execution_handler import ExecutionHandler
from bet_edge.backtesting.events import *
strategy = MACrossMACDStrategy('test_trend_strat', {}, {"ohlcv_1m" : df})
portfolio = Portfolio(initial_capital=100_000.0)
execution_handler = ExecutionHandler()

trade_log = []
# 2) Main loop
for row in strategy.data["signals_df"].iter_rows(named=True):

    market_event = MarketEvent(
            timestamp=str(row["ts_event"]), 
            bar_data=row
        )
    # Pass the MarketEvent to the Strategy
    signal_event = strategy.calculate_signals(market_event)
    if signal_event is not None:
        # Portfolio -> creates an OrderEvent
        order_event = portfolio.on_signal(signal_event)
        if order_event is not None:
            # Execution Handler -> simulates fill
            fill_event = execution_handler.execute_order(order_event, market_event)
            if fill_event is not None:
                # Update portfolio with the fill
                portfolio.on_fill(fill_event)
                # 3) Record trade/fill details
                trade_log.append({
                    "timestamp": fill_event.timestamp,
                    "symbol": fill_event.symbol,
                    "direction": fill_event.direction,
                    "quantity": fill_event.quantity,
                    "fill_price": fill_event.fill_price,
                    # You could also record PnL, commissions, etc. here
                })


# 3) Print final results
print(f"Final Equity: {portfolio.total_equity:.2f}")
print("Positions:", portfolio.positions)
print("Remaining Cash:", portfolio.current_cash)

Final Equity: 100298.25
Positions: {'ES_F': 0}
Remaining Cash: 100298.25


In [18]:
df_trades = pl.DataFrame(trade_log)
df_trades

timestamp,symbol,direction,quantity,fill_price
str,str,str,i64,f64
"""2016-06-01 07:51:00""","""ES_F""","""BUY""",1,2093.75
"""2016-06-01 07:55:00""","""ES_F""","""SELL""",1,2093.25
"""2016-06-01 15:53:00""","""ES_F""","""BUY""",1,2092.5
"""2016-06-01 16:14:00""","""ES_F""","""SELL""",1,2094.25
"""2016-06-01 18:12:00""","""ES_F""","""BUY""",1,2095.25
…,…,…,…,…
"""2025-01-07 07:20:00""","""ES_F""","""SELL""",1,6016.0
"""2025-01-07 10:14:00""","""ES_F""","""BUY""",1,6025.75
"""2025-01-07 11:26:00""","""ES_F""","""SELL""",1,6029.0


In [19]:
signals = strategy.data["signals_df"]

In [26]:
from datetime import datetime, date

df_trades.filter(pl.col("timestamp").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S").is_between(
date(2016, 6, 1),
date(2016, 6, 2),
) )

timestamp,symbol,direction,quantity,fill_price
str,str,str,i64,f64
"""2016-06-01 07:51:00""","""ES_F""","""BUY""",1,2093.75
"""2016-06-01 07:55:00""","""ES_F""","""SELL""",1,2093.25
"""2016-06-01 15:53:00""","""ES_F""","""BUY""",1,2092.5
"""2016-06-01 16:14:00""","""ES_F""","""SELL""",1,2094.25
"""2016-06-01 18:12:00""","""ES_F""","""BUY""",1,2095.25
"""2016-06-01 18:18:00""","""ES_F""","""SELL""",1,2097.5
